In [1]:
import sagemaker
sess=sagemaker.Session(default_bucket="sagemaker-us-east-1-470086202700")
role= sagemaker.get_execution_role()
account =sess.boto_session.client('sts').get_caller_identity()['Account']
region= sess.boto_session.region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
print(sess.default_bucket())

sagemaker-us-east-1-470086202700
sagemaker-us-east-1-470086202700


In [3]:
print(role)

arn:aws:iam::470086202700:role/sagemaker-developers
arn:aws:iam::470086202700:role/sagemaker-developers


## Segmentation Docker build

In [4]:
!cat Dockerfile

FROM nvcr.io/nvidia/pytorch:23.12-py3 as build

RUN apt update && DEBIAN_FRONTEND="noninteractive" apt -y install tzdata && \
    apt install -y zip libgl1-mesa-glx netbase libopencv-dev libopenblas-dev nginx && \
    apt clean && rm -rf /var/lib/apt/lists/*

RUN curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py && \
    python get-pip.py && \
    python -m pip install pip==24.2 --no-cache-dir

COPY ./scripts /opt/program

# Set work directory and install Python dependencies
WORKDIR /opt/program
## specific torch and torchvision version
RUN pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 --extra-index-url https://download.pytorch.org/whl/cu113 --no-cache-dir

RUN pip install -r requirements.txt --no-cache-dir

# Set environment variables
ENV PYTHONUNBUFFERED=TRUE \
    PYTHONDONTWRITEBYTECODE=TRUE \
    PATH="/opt/program:${PATH}"

RUN chmod +x /opt/program/serveFROM nvcr.io/nvidia/pytorch:23.12-py3 as build

RUN apt update && DEBIAN_FRONTEND="noninteractive" apt -y ins

In [5]:
repo_name="detectron2_segmentation"
base_job_name=image_tag=repo_name
%env dockerfile Dockerfile

env: dockerfile=Dockerfile
env: dockerfile=Dockerfile


### Below cells sets up enironment variables build dokcer image and push to ECR If the image is already in ECR skip these steps

In [6]:
%env account {account}
%env region {region}
%env repo_name {repo_name}
%env image_tag {image_tag}

env: account=470086202700
env: region=us-east-1
env: repo_name=detectron2_segmentation
env: image_tag=detectron2_segmentation
env: account=470086202700
env: region=us-east-1
env: repo_name=detectron2_segmentation
env: image_tag=detectron2_segmentation


In [7]:
%%sh

aws ecr describe-repositories --repository-names $repo_name > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name $repo_name > /dev/null
fi

$(aws ecr get-login --region $region --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store
https://docs.docker.com/engine/reference/commandline/login/#credentials-store




Login Succeeded
Login Succeeded


In [12]:
!docker build -t $image_tag -f $dockerfile .

Sending build context to Docker daemon  2.208MB
Step 1/9 : FROM nvcr.io/nvidia/pytorch:23.12-py3 as build
 ---> 1cff6923bda0
Step 2/9 : RUN apt update && DEBIAN_FRONTEND="noninteractive" apt -y install tzdata &&     apt install -y zip libgl1-mesa-glx netbase libopencv-dev libopenblas-dev nginx &&     apt clean && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 694e5d44f6dc
Step 3/9 : RUN curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py &&     python get-pip.py &&     python -m pip install pip==24.2 --no-cache-dir
 ---> Using cache
 ---> b2061433e076
Step 4/9 : COPY ./scripts /opt/program
Sending build context to Docker daemon  2.208MB
Step 1/9 : FROM nvcr.io/nvidia/pytorch:23.12-py3 as build
 ---> 1cff6923bda0
Step 2/9 : RUN apt update && DEBIAN_FRONTEND="noninteractive" apt -y install tzdata &&     apt install -y zip libgl1-mesa-glx netbase libopencv-dev libopenblas-dev nginx &&     apt clean && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 694e5d44f6dc
Step 3/9 : RUN

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/facebookresearch/detectron2.git (to revision bcfd464) to /tmp/pip-req-build-v8fndzz7
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-v8fndzz7
  Running command git checkout -q bcfd464
  Running command git checkout -q bcfd464
  Resolved https://github.com/facebookresearch/detectron2.git to commit bcfd464
  Preparing metadata (setup.py): started
  Resolved https://github.com/facebookresearch/detectron2.git to commit bcfd464
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): finished with status 'done'


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 322.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 365.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 322.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 365.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 590.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 412.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 590.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 412.2 MB/s eta 0:00:00
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=874a39c54cef8ea2e8768b9efce1ceb7bce11b997e4283cd79f90b66bcb99cba
  Stored in directory: /tmp/pip-ephem-wheel-cache-yczmorui/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.w

  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=43f92f36981940e900ef2bc279907997de446f03d5f8df3de80ea60c970f9c53
  Stored in directory: /tmp/pip-ephem-wheel-cache-yczmorui/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built fvcore matplotlib detectron2 antlr4-python3-runtime
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=43f92f36981940e900ef2bc279907997de446f03d5f8df3de80ea60c970f9c53
  Stored in directory: /tmp/pip-ephem-wheel-cache-yczmorui/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built fvcore matplotlib detectron2 antlr4-python3-runtime
  Attempting uninstall: scipy
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
    Uninstalling scipy-1.11.4:
      Successfully uninsta

In [13]:
!docker tag $image_tag $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

In [14]:
 !docker images

REPOSITORY                                                             TAG         IMAGE ID       CREATED        SIZE
detectron2_segmentation                                                latest      2b29cc7a70e3   1 second ago   26.7GB
470086202700.dkr.ecr.us-east-1.amazonaws.com/detectron2_segmentation   latest      2b29cc7a70e3   1 second ago   26.7GB
470086202700.dkr.ecr.us-east-1.amazonaws.com/detectron2_segmentation   <none>      7fe0d91ed0a8   18 hours ago   26.7GB
<none>                                                                 <none>      6e52b20deb2d   18 hours ago   26.5GB
nvcr.io/nvidia/pytorch                                                 23.12-py3   1cff6923bda0   8 months ago   21.9GB
REPOSITORY                                                             TAG         IMAGE ID       CREATED        SIZE
detectron2_segmentation                                                latest      2b29cc7a70e3   1 second ago   26.7GB
470086202700.dkr.ecr.us-east-1.amazonaws.com

In [17]:
 !docker push $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

The push refers to repository [470086202700.dkr.ecr.us-east-1.amazonaws.com/detectron2_segmentation]

c1268349: Preparing 
7b983b1d: Preparing 
2d55317e: Preparing 
f28f16b1: Preparing 
65b9ea06: Preparing 
e64e389c: Preparing 
47f57c32: Preparing 
4b1505f6: Preparing 
220ff626: Preparing 
47f57c32: Waiting g 
6890015a: Waiting g 
d41b5bf7: Preparing 
38864767: Preparing 
d41b5bf7: Waiting g 
38864767: Waiting g 
a3f573fd: Preparing 
54f6d038: Waiting g 
a3f573fd: Waiting g 
a0126c98: Preparing 
db49e47c: Waiting g 
a0126c98: Waiting g 
bf18a086: Waiting g 
cd9dc845: Waiting g 
8b0ad8c5: Preparing 
a6e8e18e: Preparing 
64dae3b9: Preparing 
ed62cb9b: Preparing 
7dc31822: Waiting g 
a6e8e18e: Waiting g 
ed62cb9b: Waiting g 
9441e172: Waiting g 
41d41322: Waiting g 
64dae3b9: Waiting g 
1bbe2136: Waiting g 
ecdb7186: Preparing 
5eb74a10: Preparing 
a256c52f: Preparing 
7f78ec66: Preparing 
e7459fb8: Preparing 
4edc1633: Preparing 
39e6c8e0: Preparing 
b1d479d5: Preparing 
7f78ec66: Waitin